In [1]:
!pip -q install spacy
!python -m spacy download pl_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 65.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import spacy

nlp = spacy.load("pl_core_news_sm")

df = pd.read_csv("dane1.csv")
df.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Tue Jun 01 05:26:49 +0000 2021,NaN,https://twitter.com/MarkonMarek/status/1399598...,NaN,1,1399598285082476544,beata_skwarska,1.399598e+18,8.241523e+17,...,1468,1950,7,"Warszawa, Polska",MarKon#SilniRazem ***** ***✌️✌️✌️✌️✌️,MarkonMarek,85125,NaN,NaN,False
1,NaN,Tue Jun 01 04:15:56 +0000 2021,NaN,NaN,NaN,0,1399580447902994435,NaN,NaN,NaN,...,9384,4615,17,NaN,Józef Nowacki,21lupus,74846,NaN,NaN,False
2,NaN,Tue Jun 01 04:47:19 +0000 2021,NaN,NaN,https://ift.tt/3yTNxU8,1,1399588347358633987,NaN,NaN,NaN,...,30608,105,140,"Warszawa, Polska",WTP Warszawa,WTP_Warszawa,35367,NaN,http://www.wtp.waw.pl,False
3,NaN,Tue Jun 01 05:47:47 +0000 2021,NaN,NaN,NaN,0,1399603562343194625,LukaszKohut,1.399475e+18,4.267734e+08,...,688,1374,13,NaN,marcin (m³) 🇵🇱,m3_ememem,68546,NaN,NaN,False
4,NaN,Tue Jun 01 05:03:08 +0000 2021,WspieramyRodziny DzieńDziecka,https://twitter.com/j_wisniewska/status/139959...,http://m.in,54,1399592325324619780,NaN,NaN,NaN,...,23054,678,233,"województwo śląskie, PL",Jadwiga Wiśniewska,j_wisniewska,10483,NaN,http://www.jadwigawisniewska.pl,True


In [11]:
sample_text = df["text"].dropna().astype(str).iloc[0]
sample_text

'@beata_skwarska Warszawa 😀 https://t.co/W7BcySyuNd'

In [12]:
doc = nlp(sample_text)
[(ent.text, ent.label_) for ent in doc.ents]

[('@beata_skwarska', 'placeName')]

In [13]:
def extract_entities(text):
  if not isinstance(text, str) or not text.strip():
    return [], [], []

  doc = nlp(text)

  persons = []
  places = []
  orgs = []

  #najczestsze etykiety spotykane w modelach
  person_labels   = {"persName", "PERSON", "PER"}
  place_labels    = {"placeName", "LOC", "GPE"}
  org_labels      = {"orgName", "ORG"}

  for ent in doc.ents:
    if ent.label_ in person_labels:
      persons.append(ent.text.strip())
    elif ent.label_ in place_labels:
      places.append(ent.text.strip())
    elif ent.label_ in org_labels:
      orgs.append(ent.text.strip())

  #usunecie duplikatow
  def dedupe(seq):
    seen = set()
    out = []
    for x in seq:
      if x not in seen:
        out.append(x)
        seen.add(x)
    return out

  return dedupe(persons), dedupe(places), dedupe(orgs)

In [14]:
persons_list = []
places_list = []
orgs_list = []

for t in df["text"].fillna("").astype(str):
  p, pl, o = extract_entities(t)
  persons_list.append(p)
  places_list.append(pl)
  orgs_list.append(o)

df["persons"]          = persons_list
df["places"]           = places_list
df["organizations"]    = orgs_list

df[["text", "persons", "places", "organizations"]].head(10)

,text,persons,places,organizations
0,@beata_skwarska Warszawa 😀 https://t.co/W7BcyS...,[],[@beata_skwarska],[]
1,Nieznani sprawcy podpalili kapliczkę nadrzewną...,[],"[warszawskim, Grochowie]",[]
2,⚠️ Utrudnienia w komunikacji: L20 https://t.co...,[],[Utrudnienia],[]
3,@LukaszKohut @moanrosa @LincaAgata @jan_jozef_...,[],"[Warszawa, polskiego, Warschau]",[]
4,"Dzieci to największy skarb, o który musimy dba...",[Franio],[Warszawa],[]
5,Zbliża się 81 rocznica I Transportu do KL Ausc...,[Wojciecha Korkucia],[Warszawa],"[I Transportu do KL Auschwitz, Społeczny Komit..."
6,1 czerwca 1895 urodził się Tadeusz Komorowski ...,"[Tadeusz Komorowski, Gdybyśmy]","[RP, Warszawa]","[Naczelny Wódz Polskich Sił Zbrojnych, Komenda..."
7,@BIPolska alez to brzydkie😐. Nijakie. Warszawa...,[],"[Nijakie, Warszawa]",[]
8,"Pomimo malejącej liczby urodzeń w Warszawie, l...",[],[Warszawie],[Warszawa]
9,@arayoff warszawa,[],[],[]


In [7]:
df.to_csv("dane1_part3_ner.csv", index = False)

from google.colab import files
files.download("dane1_part3_ner.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>